In [1]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis\\Research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Desktop\\Project Sem 2\\Data viz Project 2\\WellnessAI-Interactive-Health-Risk-Analysis'

In [5]:
train_data_path = "Dataset/Data_transformation/train.csv"
test_data_path = "Dataset/Data_transformation/test.csv"
target_col = "Diabetes_012"
dir_path = "Dataset/Model_selection"

In [7]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, BatchNormalization, Dropout, Concatenate
from tensorflow.keras.utils import to_categorical
from ml_Project import logger
from ml_Project.utils.common import create_directories

class ModelTrainer:
    def __init__(self, train_data_path, test_data_path, target_col, dir_path):
        self.train_data_path = train_data_path
        self.test_data_path = test_data_path
        self.target_col = target_col
        self.dir_path = dir_path

    def create_dir(self):
        create_directories([self.dir_path])

    def load_data(self):
        train_data = pd.read_csv(self.train_data_path)
        test_data = pd.read_csv(self.test_data_path)
        logger.info("Data is loaded.")
        return train_data, test_data

    def preprocess_data(self, train_data, test_data):
        X_train = train_data.drop([self.target_col], axis=1)
        X_test = test_data.drop([self.target_col], axis=1)
        y_train = train_data[[self.target_col]]
        y_test = test_data[[self.target_col]]

        # One-hot encode the target variable
        y_train = to_categorical(y_train, num_classes=3)
        y_test = to_categorical(y_test, num_classes=3)

        logger.info("Preprocessing done.")
        return X_train, X_test, y_train, y_test

    def build_custom_model(self, input_shape):
        # Input layer
        inputs = Input(shape=input_shape)

        # RNN Layer
        x = LSTM(128, return_sequences=True)(inputs)
        x = LSTM(64)(x)

        # Fully connected layers
        x = Dense(128, activation="relu")(x)
        outputs = Dense(3, activation="softmax")(x)

        # Build model
        model = Model(inputs=inputs, outputs=outputs)
        model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

        return model

    def train(self):
        self.create_dir()

        # Load data
        train_data, test_data = self.load_data()

        # Preprocess data
        X_train, X_test, y_train, y_test = self.preprocess_data(train_data, test_data)

        X_train = X_train.values.reshape(-1, X_train.shape[1], 1)
        X_test = X_test.values.reshape(-1, X_test.shape[1], 1)

        # Build and train model
        model = self.build_custom_model(X_train.shape[1:])
        model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data=(X_test, y_test))

        # Evaluate model
        loss, accuracy = model.evaluate(X_test, y_test)
        logger.info(f"Model Accuracy: {accuracy:.4f}")

        # Save model weights
        model.save_weights(f"{self.dir_path}/model_rnn.h5")
        logger.info("Model saved.")

# Usage
trainer = ModelTrainer(train_data_path, test_data_path, target_col, dir_path)
trainer.train()


[2024-08-15 19:15:40,324: INFO: common: created directory at: Dataset/Model_selection]
[2024-08-15 19:15:41,322: INFO: 1886888473: Data is loaded.]
[2024-08-15 19:15:41,354: INFO: 1886888473: Preprocessing done.]
Epoch 1/10
2849/2849 [==============================] - 135s 46ms/step - loss: 0.9630 - accuracy: 0.5066 - val_loss: 0.8730 - val_accuracy: 0.5706
Epoch 2/10
2849/2849 [==============================] - 130s 46ms/step - loss: 0.8021 - accuracy: 0.6060 - val_loss: 0.7456 - val_accuracy: 0.6341
Epoch 3/10
2849/2849 [==============================] - 130s 46ms/step - loss: 0.7498 - accuracy: 0.6318 - val_loss: 0.7268 - val_accuracy: 0.6430
Epoch 4/10
2849/2849 [==============================] - 131s 46ms/step - loss: 0.7328 - accuracy: 0.6408 - val_loss: 0.7265 - val_accuracy: 0.6363
Epoch 5/10
2849/2849 [==============================] - 131s 46ms/step - loss: 0.7165 - accuracy: 0.6499 - val_loss: 0.7238 - val_accuracy: 0.6440
Epoch 6/10
2849/2849 [==============================